In [ ]:
import numpy as np
from functools import partial
import matplotlib.pyplot as plt
import looptools.loopmath as lm
from looptools.plots import default_rc
from looptools.loop import LOOP
from looptools.components import PIControllerComponent, DoubleIntegratorComponent, PIIControllerComponent

plt.rcParams.update(default_rc)

In [ ]:
# Loop sample rate
sps = 80e6

# Define gain parameters
Kp_log2 = -10
Ki_log2 = -15
Kii_log2 = -20

# Double integrator extrapolation settings
extrapolate = (True, 1)

# Frequency range for Bode plot
frfr = np.logspace(np.log10(1e-6), np.log10(sps/2), int(1e5))
frfr = frfr[0:-1]

# Define controllers
pi = PIControllerComponent("P", sps, Kp_log2, -np.inf)
ii = DoubleIntegratorComponent("Double I", sps, Ki_log2, Kii_log2, extrapolate)
pii_1 = pi + ii
pii_2 = PIIControllerComponent("PII", sps, Kp_log2, Ki_log2, Kii_log2, extrapolate)

# Build composite loop
loop1 = LOOP(sps, [pii_1])
loop2 = LOOP(sps, [pii_2])

# Plot Bode diagrams
fig, axes = loop1.bode_plot(frfr, label='P + Double I')
loop2.bode_plot(frfr, axes=axes, label='PII', ls='--')
plt.show()

In [ ]:
# Plot Nyquist diagrams
fig, ax = loop1.nyquist_plot(frfr, label='P + Double I', logy=True, logx=True, alpha=0.5, critical_point=True)
loop2.nyquist_plot(frfr, ax=ax, label='PII', ls=':')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from looptools.component import Component
from looptools.components import PIIControllerComponent
import looptools.loopmath as lm

def test_gain_for_crossover_frequency():
    # Sampling rate and frequency array
    sps = 80e6
    frfr = np.logspace(np.log10(1e0), np.log10(40e6), int(1e5))[:-1]

    # P gain (dB → log2)
    p_log2_gain = lm.db_to_log2_gain(60)

    # Test crossover frequencies for I and II blocks
    i_crossovers = [10e3, 100e3, 1e6]
    ii_crossovers = [10, 100, 1000]

    # ---------- Test P+I Loop ----------
    fig_i, ax_i = plt.subplots()
    for f_cross in i_crossovers:
        ki_log2_gain = lm.gain_for_crossover_frequency(p_log2_gain, sps, f_cross, kind='I')
        controller = PIIControllerComponent(f"P+I @ {f_cross/1e3:.0f}kHz", sps=sps,
                                            Kp=p_log2_gain, Ki=ki_log2_gain, Kii=-60)
        mag, phase = controller.bode(frfr, dB=True)
        ax_i.semilogx(frfr, mag, label=f"I @ {f_cross/1e3:.0f} kHz")

    ax_i.set_title("P+I Test")
    ax_i.set_xlabel("Frequency [Hz]")
    ax_i.set_ylabel("Magnitude [dB]")
    ax_i.legend()
    ax_i.grid(True, which='both', ls='--')

    # # ---------- Test II Block ----------
    fig_ii, ax_ii = plt.subplots()
    for f_cross in ii_crossovers:
        # Use dummy I crossover to allow II gain computation; ignore returned Ki
        _, kii_log2_gain = lm.gain_for_crossover_frequency(p_log2_gain, sps, (1e-6, f_cross), kind='II')
        controller = PIIControllerComponent(f"P+II @ {f_cross:.0f}Hz", sps=sps,
                                            Kp=p_log2_gain, Ki=-60, Kii=kii_log2_gain,
                                            extrapolate=(False, f_cross/10))
        mag, phase = controller.bode(frfr, dB=True)
        ax_ii.semilogx(frfr, mag, label=f"II @ {f_cross:.0f} Hz")

    ax_ii.set_title("P+II Test")
    ax_ii.set_xlabel("Frequency [Hz]")
    ax_ii.set_ylabel("Magnitude [dB]")
    ax_ii.legend()
    ax_ii.grid(True, which='both', ls='--')

    # ---------- Test P+I+II Loop ----------
    fig_pii, ax_pii = plt.subplots()
    for i_cross, ii_cross in zip(i_crossovers, ii_crossovers):
        ki_log2_gain, kii_log2_gain = lm.gain_for_crossover_frequency(p_log2_gain, sps, (i_cross, ii_cross), kind='II')
        controller = PIIControllerComponent(f"P+I+II @ {i_cross/1e3:.0f}kHz/{ii_cross:.0f}Hz", sps=sps,
                                            Kp=p_log2_gain, Ki=ki_log2_gain, Kii=kii_log2_gain,
                                            extrapolate=(False, ii_cross/10))
        mag, phase = controller.bode(frfr, dB=True)
        ax_pii.semilogx(frfr, mag, label=f"I={i_cross/1e3:.0f}kHz, II={ii_cross:.0f}Hz")

    ax_pii.set_title("P+I+II Test")
    ax_pii.set_xlabel("Frequency [Hz]")
    ax_pii.set_ylabel("Magnitude [dB]")
    ax_pii.grid(True, which='both', ls='--')
    ax_pii.legend()

    plt.show()

# Run the test
test_gain_for_crossover_frequency()

In [ ]:
def plot_pii_component_decomposition(controller, frfr, dB=True, label_suffix=""):
    """
    Plot the Bode magnitude of the P, I, II components and their total sum.

    Parameters
    ----------
    controller : PIIControllerComponent
        The controller instance (assumes it was built with Kp, Ki, Kii).
    frfr : array_like
        Frequency array (Hz).
    dB : bool, optional
        Plot magnitude in dB (default: True).
    label_suffix : str
        Suffix label for disambiguating multiple plots.
    """
    # Reconstruct individual components (matches structure in __init__)
    Kp = controller.Kp
    Ki = controller.Ki
    Kii = controller.Kii
    sps = controller.sps
    extrapolate = controller.extrapolate

    P = Component("P", sps, np.array([Kp]), np.array([1.0]))
    I = Component("I", sps, np.array([Ki]), np.array([1.0, -1.0]))
    II = Component("II", sps, np.array([Kii]), np.array([1.0, -2.0, 1.0]))
    II.TF = partial(II.TF, extrapolate=extrapolate[0], f_trans=extrapolate[1], power=-2)

    # Compute magnitudes
    f = np.asarray(frfr)
    P_mag = np.abs(P.TF(f=f))
    I_mag = np.abs(I.TF(f=f))
    II_mag = np.abs(II.TF(f=f))
    total_mag = np.abs(P.TF(f=f) + I.TF(f=f) + II.TF(f=f))

    if dB:
        P_mag = 20 * np.log10(P_mag)
        I_mag = 20 * np.log10(I_mag)
        II_mag = 20 * np.log10(II_mag)
        total_mag = 20 * np.log10(total_mag)

    # Plot
    fig, ax = plt.subplots()
    ax.semilogx(f, P_mag, label="P" + label_suffix, color='gray')
    ax.semilogx(f, I_mag, label="I" + label_suffix, color='tomato')
    ax.semilogx(f, II_mag, label="II" + label_suffix, color='lime')
    ax.semilogx(f, total_mag, label="P+I+II" + label_suffix, lw=2, color='deepskyblue', ls='--')
    ax.set_xlim(f[0], f[-1])

    ax.set_title("Component Decomposition")
    ax.set_xlabel("Frequency [Hz]")
    ax.set_ylabel("Magnitude [dB]" if dB else "Magnitude")
    ax.grid(True, which='both', ls='--')
    ax.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
Kp=lm.db_to_log2_gain(60)
Ki, Kii = lm.gain_for_crossover_frequency(Kp, sps, (1e3,1e1),  kind='II')

sps = 80e6
frfr = np.logspace(np.log10(1e0), np.log10(1e4), int(1e5))[:-1]

controller = PIIControllerComponent(name='Controller', sps=sps, Kp=lm.db_to_log2_gain(60), Ki=Ki, Kii=Kii)
plot_pii_component_decomposition(controller, frfr)